# Subsequent Memory Effect

### Imports

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm


### Settings

In [2]:
# Paths
PROJECT_PATH = 'C:/Users/micha/projects/oscillation_vs_exponent/'


### Load data

In [3]:
# load spectral parameterization results
df = pd.read_csv(f"{PROJECT_PATH}/data/results/spectral_parameters.csv", index_col=0)

# show
print(df.shape)
print(len(df)/8)
df.head()

(5560, 27)
695.0


,index,patient,chan_idx,label,pos_y,pos_x,pos_z,material,memory,epoch,...,gamma_pw,gamma_bw,error,r_squared,r2_adj,f_rotation,alpha,alpha_adj,gamma,gamma_adj
0,9,pat02,0,A01-A02,-65.43100,61.94490,3.55955,words,hit,pre,...,NaN,NaN,0.069006,0.992773,0.992538,46.908382,4.439106,3.976722,1.947740,1.077783
1,10,pat02,1,A02-A03,-70.93895,57.17765,12.15540,words,hit,pre,...,NaN,NaN,0.062446,0.991529,0.991252,29.941520,4.148422,3.611782,1.980923,1.074298
2,11,pat02,2,A03-A04,-75.39550,51.39440,20.94335,words,hit,pre,...,NaN,NaN,0.054597,0.992668,0.992428,19.961014,4.169199,3.728224,1.984640,0.881316
3,12,pat02,3,A04-A05,-78.91950,43.90980,30.13485,words,hit,pre,...,0.249224,2.712309,0.056073,0.992178,0.991923,49.902534,4.362397,3.488728,2.267206,1.267119
4,13,pat02,4,A05-A06,-80.96735,35.21485,38.13475,words,hit,pre,...,NaN,NaN,0.069188,0.989511,0.989169,59.883041,4.332351,3.725463,2.185415,1.443403


In [4]:
# compute stimulus-evoked change in each parameter

def load_params():
    # load data
    df = pd.read_csv(f"{PROJECT_PATH}/data/results/spectral_parameters.csv", index_col=0)
    df = df[['patient', 'chan_idx', 'material', 'memory', 'epoch', 'exponent', 
             'alpha', 'alpha_adj', 'gamma', 'gamma_adj']] # drop unneeded columns   
    # df = df.loc[df['memory']=='hit'] # drop unsuccessful trials

    # pivot table (based on epoch)
    index = ['patient', 'chan_idx', 'material', 'memory']
    features = ['exponent', 'alpha', 'alpha_adj', 'gamma', 'gamma_adj']
    df = df.pivot_table(index=index, columns='epoch', values=features).reset_index()

    # compute difference in parameters
    for feature in features:
        df[f"{feature}_diff"] = df[(feature, 'post')] - df[(feature, 'pre')]

    # drop original columns
    df = df.drop(columns=[(feature, epoch) for feature in features for epoch in ['pre', 'post']])
    df.columns = df.columns.droplevel(1)

    # drop diff suffix
    df.columns = [col.replace('_diff', '') for col in df.columns]

    return df

df_diff = load_params()
df_diff

,patient,chan_idx,material,memory,exponent,alpha,alpha_adj,gamma,gamma_adj
0,pat02,0,faces,hit,-0.026946,-0.201906,-1.248484,0.030402,0.185490
1,pat02,0,faces,miss,-0.240082,-0.045629,-0.150026,-0.013548,0.136078
2,pat02,0,words,hit,0.075373,-0.005121,-0.141448,-0.026005,-0.328046
3,pat02,0,words,miss,0.104594,-0.024285,-0.418910,-0.059830,0.133254
4,pat02,1,faces,hit,-0.202433,-0.283929,-0.601808,-0.009944,-0.123020
...,...,...,...,...,...,...,...,...,...
2675,pat22,51,words,miss,-0.367073,-0.045391,NaN,0.214991,-0.019540
2676,pat22,52,faces,hit,-0.171604,0.165392,-0.075834,0.044464,-0.267102
2677,pat22,52,faces,miss,-0.192198,0.056379,0.036484,0.038762,-0.190946
2678,pat22,52,words,hit,-0.346883,-0.021038,0.151997,0.193621,0.294967


In [5]:
# marge with task-modulation results

# load task-modulation results
df_tm = pd.read_csv(r"C:\Users\micha\projects\oscillation_vs_exponent\data\results\ieeg_modulated_channels.csv", index_col=0)

# join
df_merge = pd.merge(df_diff, df_tm, on=['patient','chan_idx'])

df_merge

,patient,chan_idx,material,memory,exponent,alpha,alpha_adj,gamma,gamma_adj,sig_alpha,sig_gamma,sig_all,sig_any
0,pat02,0,faces,hit,-0.026946,-0.201906,-1.248484,0.030402,0.185490,False,False,False,False
1,pat02,0,faces,miss,-0.240082,-0.045629,-0.150026,-0.013548,0.136078,False,False,False,False
2,pat02,0,words,hit,0.075373,-0.005121,-0.141448,-0.026005,-0.328046,False,False,False,False
3,pat02,0,words,miss,0.104594,-0.024285,-0.418910,-0.059830,0.133254,False,False,False,False
4,pat02,1,faces,hit,-0.202433,-0.283929,-0.601808,-0.009944,-0.123020,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2675,pat22,51,words,miss,-0.367073,-0.045391,NaN,0.214991,-0.019540,False,False,False,False
2676,pat22,52,faces,hit,-0.171604,0.165392,-0.075834,0.044464,-0.267102,True,False,False,True
2677,pat22,52,faces,miss,-0.192198,0.056379,0.036484,0.038762,-0.190946,True,False,False,True
2678,pat22,52,words,hit,-0.346883,-0.021038,0.151997,0.193621,0.294967,True,False,False,True


### Run 2x2 Anova

In [7]:
# drop missing data channels

# drop NaN
df_merge['drop'] = False
for i_row, row in df_merge.iterrows():
    if ((np.isnan(row['exponent'])) | (np.isnan(row['alpha'])) | (np.isnan(row['alpha_adj'])) | (np.isnan(row['gamma'])) | (np.isnan(row['gamma_adj']))):
        df_merge.loc[((df_merge['patient']==row['patient']) & (df_merge['chan_idx']==row['chan_idx'])), 'drop'] = True
df_clean = df_merge.loc[~df_merge['drop']]
print(f"NaN channels to drop: {int(np.sum(df_merge['drop'])/4)}")

# add unique_id column
for i_row, row in df_clean.iterrows():
    df_clean.loc[i_row, 'unique_id'] = f"{row['patient']}_{row['chan_idx']}"

# drop rows that are not represented in each conditions
for id in df_clean['unique_id'].unique():
    if len(df_clean.loc[df_clean['unique_id']==id]) != 4:
        df_clean = df_clean.loc[df_clean['unique_id']!=id]
        print(f"dropped {id}")

df_clean


NaN channels to drop: 172


C:\Users\micha\AppData\Local\Temp\ipykernel_84152\1478057283.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.loc[i_row, 'unique_id'] = f"{row['patient']}_{row['chan_idx']}"


dropped pat04_28
dropped pat04_68
dropped pat04_69
dropped pat08_37
dropped pat08_38
dropped pat08_39
dropped pat15_52
dropped pat15_53
dropped pat20_4


,patient,chan_idx,material,memory,exponent,alpha,alpha_adj,gamma,gamma_adj,sig_alpha,sig_gamma,sig_all,sig_any,drop,unique_id
0,pat02,0,faces,hit,-0.026946,-0.201906,-1.248484,0.030402,0.185490,False,False,False,False,False,pat02_0
1,pat02,0,faces,miss,-0.240082,-0.045629,-0.150026,-0.013548,0.136078,False,False,False,False,False,pat02_0
2,pat02,0,words,hit,0.075373,-0.005121,-0.141448,-0.026005,-0.328046,False,False,False,False,False,pat02_0
3,pat02,0,words,miss,0.104594,-0.024285,-0.418910,-0.059830,0.133254,False,False,False,False,False,pat02_0
4,pat02,1,faces,hit,-0.202433,-0.283929,-0.601808,-0.009944,-0.123020,False,False,False,False,False,pat02_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2659,pat22,47,words,miss,0.032693,0.196230,-0.430461,-0.013557,0.022321,False,False,False,False,False,pat22_47
2660,pat22,48,faces,hit,-0.110476,0.021113,0.075012,0.005516,-0.041368,False,False,False,False,False,pat22_48
2661,pat22,48,faces,miss,-0.243923,0.023257,-0.470580,0.072559,0.163881,False,False,False,False,False,pat22_48
2662,pat22,48,words,hit,-0.256222,-0.003696,0.180704,-0.026619,-0.468044,False,False,False,False,False,pat22_48


In [19]:
# get task-modulated channels
df_sig = df_clean.loc[df_clean['sig_all']]
print(f"N task-modulated channels: \t{int(len(df_sig)/4)}")


N task-modulated channels: 	30


In [20]:
# run 2-way anova (memory x material)

# loop over variables of interest
results = {}
for var in ['exponent', 'alpha', 'alpha_adj', 'gamma', 'gamma_adj']:
    # repeated measures anova
    results_i = sm.stats.AnovaRM(df_sig, f"{var}", 'unique_id', 
                                 within=['material', 'memory'], aggregate_func='mean'
                                 ).fit()

    # aggregate
    results[var] = results_i

    # show results
    print(f"\n\n===================  {var}  ===================\n")
    print(results_i)



===================  exponent  ===================

                    Anova
                F Value Num DF  Den DF Pr > F
---------------------------------------------
material         8.6416 1.0000 29.0000 0.0064
memory           0.0738 1.0000 29.0000 0.7878
material:memory  2.4513 1.0000 29.0000 0.1283



===================  alpha  ===================

                    Anova
                F Value Num DF  Den DF Pr > F
---------------------------------------------
material         1.1281 1.0000 29.0000 0.2969
memory           1.5514 1.0000 29.0000 0.2229
material:memory  0.1768 1.0000 29.0000 0.6773



===================  alpha_adj  ===================

                    Anova
                F Value Num DF  Den DF Pr > F
---------------------------------------------
material         8.4263 1.0000 29.0000 0.0070
memory           1.8103 1.0000 29.0000 0.1889
material:memory  0.8436 1.0000 29.0000 0.3659



===================  gamma  ===================

                  

In [22]:
features =['material', 'memory', 'exponent', 'alpha', 'alpha_adj', 'gamma', 'gamma_adj']
df_mean = df_clean.loc[:, features].groupby(by=['material','memory']).mean()
df_mean

exponent     alpha  alpha_adj     gamma  gamma_adj
material memory                                                    
faces    hit    -0.077380 -0.029447  -0.053579  0.016574   0.018134
         miss   -0.072055 -0.031970  -0.053948  0.018081   0.040640
words    hit    -0.079140 -0.022592  -0.022053  0.011911   0.024407
         miss   -0.078280 -0.032965  -0.027778  0.010101   0.013005